In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_parquet('/kaggle/input/2024ucs654labeval1007/Lab Eval/train_data.parquet').sample(frac=0.8, random_state=42)
df.shape

In [ ]:
df.head()

In [ ]:
test = pd.read_parquet('/kaggle/input/2024ucs654labeval1007/Lab Eval/test_data.parquet')
test.shape

In [ ]:
test.reset_index(inplace=True)
df.reset_index(inplace=True)

In [ ]:
ids = test['id']
test = test.drop('data_type', axis = 1)

In [ ]:
df = df.drop(['id','era','data_type','T0','T1','T2','T3','T4','T5','T6','T8','T9'], axis=1)
#df = df.drop('data_type', axis=1)

In [ ]:
numeric_columns = df.select_dtypes(include=['int8']).columns
len(numeric_columns)

In [ ]:
df[numeric_columns] = df[numeric_columns].astype('category')

In [ ]:
X = df.drop('T7', axis=1)
y = df['T7']

correlation_values = X.apply(lambda x: x.corr(y))
    
print(correlation_values)

In [ ]:
correlation_values = correlation_values.sort_values()
print(correlation_values)

In [ ]:
cor = correlation_values[correlation_values<0.00]
cor2 = correlation_values[correlation_values>=0.00]
print(cor.index)
print(cor2.index)

In [ ]:
fincols = cor2.index

In [ ]:
t7 = df['T7']
df = df[fincols]
df['T7'] = t7

In [ ]:
test = test[fincols]

In [ ]:
df['T7'].value_counts()

In [ ]:
X = df.drop('T7', axis=1)
y = df['T7']

### RANDOM Forest

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Assuming X and y are your features and target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Random Forest Regressor
rf = RandomForestRegressor(random_state=42,n_jobs=-1)

# Train the model
rf.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = rf.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)


In [ ]:
pred_test = rf.predict(test)

sub = pd.DataFrame({'ID': ids, 'Target': pred_test})

sub.to_csv('submission.csv', index=False)

sub.head()